# Comienza el análisis ETL STEAM GAMES

Primero, importo las librerias necesarias para hacer el respectivo análisis.

In [1]:
import pandas as pd
import jsonlines
import gzip
import ast

Se procede a abrir los JSON

Comenzando con la data base STEAM GAMES

In [3]:
# Crear una lista para almacenar los objetos JSON
datos_json = []

# Abre el archivo JSON
with jsonlines.open('output_steam_games.json') as archivo_json:
    for objeto_json in archivo_json:
        datos_json.append(objeto_json)

# Convertir la lista de objetos JSON en un DataFrame usando Pandas.
df_steam_games = pd.DataFrame(datos_json)

df_steam_games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"


Ahora se procede a guardar en un .csv los datos STEAM GAMES

In [5]:
df_steam_games.to_csv('steam_games.csv', index=False)

Se procede a desanidar la base de datos USER_REVIEWS, leyendo un archivo comprimido con gzip, interpreta cada línea como una expresión de Python y devuelve los objetos resultantes como un generador. A continuación se hace la declaración de la función:

In [6]:
def parse (path):
    with gzip.open(path, 'r') as g:
        for l in g:
            yield eval(l)


Se ejecuta la función parse,  extrayendo los datos de un archivo JSON comprimido y los está convirtiendo en DataFrames de Pandas, que luego se almacenan en una lista para su posterior procesamiento o análisis.

In [8]:
rows = []
for line in parse('user_reviews.json.gz'):
    row = pd.json_normalize(line)
    rows.append(row)

Procedemos a concatenar las filas en una sola estructura de datos (Dataframe)

In [9]:
df = pd.concat(rows, ignore_index=True)

Se revisa como queda el DataFrame

In [10]:
df

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


El algoritmo que viene a continuación, descompone una columna que contiene listas en múltiples filas, replica las otras columnas para cada elemento de la lista, convierte la columna de listas en un DataFrame, concatena este DataFrame con el original y luego elimina la columna original de listas.

In [11]:

# Descomponer la columna "reviews" en múltiples filas
df = df.explode("reviews")

# Convertir la columna "reviews" en un DataFrame
df_reviews = df["reviews"].apply(pd.Series)

# Concatenar el DataFrame de "reviews" con el DataFrame existente
df = pd.concat([df, df_reviews], axis=1)

# Eliminar la columna original "reviews"
df.drop("reviews", axis=1, inplace=True)

Actualiza el Dataframe Reviews

In [12]:
df_reviews = df
df_reviews

,user_id,user_url,0,funny,helpful,item_id,last_edited,posted,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,NaN,,No ratings yet,1250,,"Posted November 5, 2011.",True,Simple yet with great replayability. In my opi...
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,NaN,,No ratings yet,22200,,"Posted July 15, 2011.",True,It's unique and worth a playthrough.
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,NaN,,No ratings yet,43110,,"Posted April 21, 2011.",True,Great atmosphere. The gunplay can be a bit chu...
1,js41637,http://steamcommunity.com/id/js41637,NaN,,15 of 20 people (75%) found this review helpful,251610,,"Posted June 24, 2014.",True,I know what you think when you see this title ...
1,js41637,http://steamcommunity.com/id/js41637,NaN,,0 of 1 people (0%) found this review helpful,227300,,"Posted September 8, 2013.",True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...,...,...,...,...
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,NaN,,No ratings yet,70,,Posted July 10.,True,a must have classic from steam definitely wort...
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,NaN,,No ratings yet,362890,,Posted July 8.,True,this game is a perfect remake of the original ...
25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,NaN,1 person found this review funny,1 of 2 people (50%) found this review helpful,273110,,Posted July 3.,True,had so much fun plaing this and collecting res...
25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,NaN,,No ratings yet,730,,Posted July 20.,True,:D


Guarda en un archivo tipo csv el DataFrame 'user_reviews'

In [13]:
df_reviews.to_csv('user_reviews.csv', index=False)